In [ ]:
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

from llama_index import VectorStoreIndex, SimpleDirectoryReader, load_index_from_storage, StorageContext
from IPython.display import Markdown, display

In [ ]:
# load documents
documents = SimpleDirectoryReader('../prep_materials/data').load_data()